# préparation des données
besoin de convertir les données du format tagtog vers spacy

On veut pouvoir utiliser l'objet [Scorer](https://spacy.io/api/scorer#score) qui nécessite la construction d'objets [Example](https://spacy.io/api/example).

> An Example holds the information for one training instance. It stores two Doc objects: one for holding the gold-standard reference data, and one for holding the predictions of the pipeline.

## Exemple de code de scoring avec Spacy
On obtient un F-Score : c'est parfait.

In [1]:
import spacy
from spacy.scorer import Scorer
from spacy.training.example import Example

text_and_target_entities: list[str, list[tuple[int, int, str]]] = [
    ("Qui est Grégory Doucet?",
     [(8, 22, 'PER')]),
    ("J'aime Londres et Berlin.",
     [(7, 14, 'LOC'), (18, 24, 'LOC')])
]

# Split text and target entities to enable the efficient use of the `pipe()` method.
texts, target_entities = zip(*text_and_target_entities)


def evaluate(ner_model, texts, target_entities, debug=False):
    examples = []
    pred_documents = ner_model.pipe(texts)
    for pred_doc, text, entities in zip(pred_documents, texts, target_entities):
        if debug:
            print("Pred.:", [(ent.text, ent.label_) for ent in pred_doc.ents], " ↔ Targ.:", [(text[e[0]:e[1]], e[2]) for e in entities])
        example = Example.from_dict(pred_doc, {"entities": entities})
        examples.append(example)
    
    scorer = Scorer()
    scores = scorer.score_spans(examples, "ents")
    print(scores["ents_f"])
    return scores

ner_model = spacy.load('fr_core_news_sm') # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, texts, target_entities, debug=True)
print(results)

Pred.: [('Grégory Doucet', 'PER')]  ↔ Targ.: [('Grégory Doucet', 'PER')]
Pred.: [('Londres', 'LOC'), ('Berlin', 'LOC')]  ↔ Targ.: [('Londres', 'LOC'), ('Berlin', 'LOC')]
1.0
{'ents_p': 1.0, 'ents_r': 1.0, 'ents_f': 1.0, 'ents_per_type': {'PER': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'LOC': {'p': 1.0, 'r': 1.0, 'f': 1.0}}}


## Conversion du format TagTog vers Spacy

In [2]:
from glob import glob
import os.path

PATH_JSON_FILES_DIR = "/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool"

files = sorted(glob(os.path.join(PATH_JSON_FILES_DIR, "*.json")))
print("Found", len(files), "files.")
files[:10]

Found 100 files.


['/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a.0MBIm18ieWUH8iyy4d4RIFZKSu-FRA02802_Mirbeau.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a.GwmWhiyt0OKvDrD6b._1MxEt2K-FRA03101_Ponson.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a.JUEoYT5Jt44vnpvLAg8WSk0iVm-FRA00901_Daudet.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a.n8hDHVP39RVA3a5sKkRffFXIUO-FRA04102_Erckmann.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a0z.I.nFSfzECA9d.L_J7DCWOZXm-FRA05801_Mille.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a34P3V2Cg118Ym0ahNVjTy4.Ex5O-FRA04601_Bazin.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a3MNF7TUrtea7XfRiJXn6Wk0b858-FRA07201_Montagne.txt.ann.json',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a49_fmd9IHJ2wC1gsQKizy8CDYZi-FRA0

In [3]:
sample = files[50]
sample

'/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/aS22wZszAN02LNh6VcXvYm.zrhiq-FRA05401_Kock.txt.ann.json'

In [4]:
import json

In [5]:
TAGTOG_LABELS = {
  "e_8" : "EVENT",
  "e_3" : "ORG",
  "e_2" : "LOC",
  "e_7" : "ROLE",
  "e_5" : "WORK",
  "e_6" : "DEMO",
  "e_4" : "OTHER",
  "e_1" : "PER"
}

KEEP_TAGS = ["LOC", "PER"]

In [6]:
def parse_tagtog_json(filename: str) -> list[tuple[int, int, str]]:
    data = None
    with open(filename, encoding="utf8") as in_file:
        data = json.load(in_file)
    data = data["entities"]
    results = []
    for elt in data:
        label = TAGTOG_LABELS[elt.get("classId")]
        first_offset = elt["offsets"][0]
        start = first_offset["start"]
        text = first_offset["text"]
        if label in KEEP_TAGS:
            results.append((start, start+len(text), label))
    return results

In [7]:
parse_tagtog_json(sample)

[(0, 9, 'PER'),
 (1923, 1928, 'LOC'),
 (2852, 2863, 'LOC'),
 (3431, 3458, 'LOC'),
 (4425, 4431, 'PER'),
 (5222, 5228, 'PER'),
 (5284, 5290, 'PER'),
 (5745, 5750, 'LOC'),
 (5846, 5857, 'LOC'),
 (6284, 6290, 'PER'),
 (8483, 8489, 'PER'),
 (9569, 9575, 'PER')]

## Génération de données text ↔ targets pour le jeu de données complet

In [8]:
PATH_DATASET_DIR = "/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset"

json_files = glob(os.path.join(PATH_DATASET_DIR, "annotations/pool/*.json"))
print("Found", len(json_files), "JSON files.")


text_files = glob(os.path.join(PATH_DATASET_DIR, "texts/*.txt"))
print("Found", len(text_files), "text files.")

Found 100 JSON files.
Found 100 text files.


Nous allons à présent mettre les fichiers en correspondance pour faciliter l'exploitation des annotations.

In [9]:
import re

In [10]:
# text filename format sample: "tr_FRA00502_Balzac.txt"
# base identifier (like "FRA00501_Balzac") to full filename
id_to_text_filename: dict[str, str] = {
    re.match(r".*/tr_(FRA[^/]*)\.txt", filename).group(1): filename for filename in text_files
}
id_to_text_filename

{'FRA01203_Feval': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA01203_Feval.txt',
 'FRA00701_Carraud': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00701_Carraud.txt',
 'FRA02301_Greville': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA02301_Greville.txt',
 'FRA01602_GautierJ': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA01602_GautierJ.txt',
 'FRA03401_Reybaud': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA03401_Reybaud.txt',
 'FRA07601_Leroux': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA07601_Leroux.txt',
 'FRA02901_Noailles': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA02901_Noailles.txt',
 'FRA02202_Gouraud': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA02202_Gouraud.txt',
 'FRA03704_Sand': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA03704_Sand.txt',
 'FRA00102_Adam': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FR

In [11]:
# JSON filename format sample: "aHqzKMns1U.VfQU61Eidq6joaKY8-FRA00501_Balzac.txt.ann.json"
# base identifier (like "FRA00501_Balzac") to full filename
id_to_json_filename: dict[str, str] = {
    re.match(r".*/[^/]*-(FRA[^/]*)\.txt\.ann\.json", filename).group(1): filename for filename in json_files
}
id_to_json_filename

{'FRA01302_Flaubert': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a_cdO7G8FaGY5emxh2eoIs4VTvWq-FRA01302_Flaubert.txt.ann.json',
 'FRA02701_Maupassant': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a7K9yh.2wyxHG509T8Fb8wCuufFy-FRA02701_Maupassant.txt.ann.json',
 'FRA01101_Dombre': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/aW9XeObpMHpr3FpcV0GTZMD1ntlK-FRA01101_Dombre.txt.ann.json',
 'FRA00501_Balzac': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/aHqzKMns1U.VfQU61Eidq6joaKY8-FRA00501_Balzac.txt.ann.json',
 'FRA00401_Allais': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/aCQVyShyQUhBawtxZ9Eq7whm687a-FRA00401_Allais.txt.ann.json',
 'FRA01603_GautierJ': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/a9_JuHi9In_5VsmgTMcy5iOTqgVe-FRA01603_GautierJ.txt.ann.json',
 'FRA04002_Verne': '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/abRcy3SO2ZZgE3Xq

In [12]:
# Fix inconsistent naming
id_to_json_filename["FRA06201_Viel-Castel"] = id_to_json_filename["FRA06201_Viel_Castel"]
del id_to_json_filename["FRA06201_Viel_Castel"]

In [13]:
# Merge everything
id_to_text_and_json = {
    k: (v, id_to_json_filename[k]) for (k,v) in id_to_text_filename.items()
}
len(id_to_text_and_json)

100

In [14]:
list(id_to_text_and_json.items())[:3]

[('FRA01203_Feval',
  ('/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA01203_Feval.txt',
   '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/amC03G616xHD2JuEngguPRbuYrUq-FRA01203_Feval.txt.ann.json')),
 ('FRA00701_Carraud',
  ('/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00701_Carraud.txt',
   '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/afvwbyoO7Lc3pbviRJ4voJDp9CFy-FRA00701_Carraud.txt.ann.json')),
 ('FRA02301_Greville',
  ('/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA02301_Greville.txt',
   '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/annotations/pool/abBmPweZyAxzbqSOGsC_bzk_WIuG-FRA02301_Greville.txt.ann.json'))]

On fait un export rapide et moche de toute le contenu, et on fera un README propre pour ce dérivé du jeu de donnée original.

Il faudra publier ce dérivé en [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/) conformément à la licence du [travail original](https://dspace-clarin-it.ilc.cnr.it/repository/xmlui/handle/20.500.11752/OPEN-986).

In [15]:
def read_text_file(filename: str) -> str:
    with open(filename, encoding="utf8") as in_file:
        return "".join(in_file.readlines())

In [16]:
all_data: dict[str, tuple[str, list[tuple[int, int, str]]]] = {}  # id → (texte, liste entités sous la forme (start, end, type))
for k, (text_filename, json_filename) in id_to_text_and_json.items():
    all_data[k] = (
        read_text_file(text_filename),
        parse_tagtog_json(json_filename)
    )
len(all_data)

100

In [17]:
# Fix duplicate entity which crashed spacy evaluation
sample_id = "FRA00502_Balzac"
sample_text, sample_target = all_data[sample_id]
sample_target.remove((27270, 27275, 'PER'))

In [18]:
# Fix FRA01002_DelarueMardrus where all indices are shifted left by 2 chars (Unicode BOM?)
sample_id = "FRA01002_DelarueMardrus"
sample_text, sample_target = all_data[sample_id]
new_target = []
for start, end, label in sample_target:
    new_target.append((start+2, end+2, label))
all_data[sample_id] = (sample_text, new_target)

Now try to display some content.

In [19]:
sample_id = "FRA01203_Feval"
sample_text = all_data[sample_id][0]
print(sample_text)

I -- La chanson
Il n'y a pas encore bien longtemps, le voyageur qui allait de Paris à Brest, de la capitale du royaume à la première de nos cités maritimes, s'endormait et s'éveillait deux fois, bercé par les cahots de la diligence, avant d'apercevoir les maigres moissons, les pommiers trapus et les chênes ébranlés de la pauvre Bretagne. Il s'éveillait la première fois dans les fertiles plaines du Perche, tout près de la Beauce, ce paradis des négociants en farine : il se rendormait poursuivi par l'aigrelet parfum du cidre de l'Orne et par le patois nasillard des naturels de la Basse-Normandie. Le lendemain matin, le paysage avait changé ; c'était Vitré, la gothique momie, qui penche ses maisons noires et les ruines chevelues de son château sur la pente raide de sa colline ; c'était l'échiquier de prairies plantées çà et là de saules et d'oseraies où la Vilaine plie et replie en mille détours son étroit ruban d'azur. Le ciel, bleu la veille, était devenu gris ; l'horizon avait perdu so

In [20]:
sample_target = all_data[sample_id][1]
print(sample_target)

[(78, 83, 'LOC'), (86, 91, 'LOC'), (330, 338, 'LOC'), (401, 407, 'LOC'), (425, 431, 'LOC'), (534, 538, 'LOC'), (585, 600, 'LOC'), (656, 661, 'LOC'), (863, 873, 'LOC'), (1173, 1191, 'LOC'), (1193, 1211, 'LOC'), (1351, 1363, 'PER'), (1365, 1380, 'PER'), (1382, 1398, 'PER'), (1457, 1465, 'PER'), (2355, 2373, 'LOC'), (3775, 3784, 'PER'), (3797, 3815, 'PER'), (4159, 4167, 'LOC'), (4274, 4280, 'LOC'), (4437, 4448, 'LOC'), (4460, 4466, 'LOC'), (4699, 4717, 'PER'), (4990, 4998, 'LOC'), (5015, 5023, 'PER'), (5180, 5197, 'PER'), (5199, 5209, 'PER'), (5211, 5226, 'PER'), (5266, 5276, 'LOC'), (5280, 5286, 'LOC')]


On vérifie rapidement que le format semble OK pour Spacy (et displaCy).

In [21]:
evaluate(ner_model, [sample_text], [sample_target])

0.41666666666666663


{'ents_p': 0.35714285714285715,
 'ents_r': 0.5,
 'ents_f': 0.41666666666666663,
 'ents_per_type': {'LOC': {'p': 0.5,
   'r': 0.6842105263157895,
   'f': 0.5777777777777778},
  'PER': {'p': 0.18181818181818182,
   'r': 0.18181818181818182,
   'f': 0.18181818181818182},
  'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}

In [22]:
doc = ner_model(sample_text)

In [23]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

On peut afficher la vérité terrain directement également <https://spacy.io/usage/visualizers#manual-usage>, après une petite conversion pour coller au format suivant, et en forçant le mode "manuel":
```python
{
    "text": "But Google is starting from behind.",
    "ents": [{"start": 4, "end": 10, "label": "ORG"}],
    "title": None
}
```

In [24]:
manual_content = {
    "text": sample_text,
    "ents": [{"start": e[0], "end": e[1], "label": e[2]} for e in sample_target],
    "title": sample_id
}
displacy.render(manual_content, manual=True, style="ent", jupyter=True)

## Export

In [25]:
with open("../dataset/French_ELTEC_NER_Open_Dataset.json", mode="w", encoding="utf8") as out_file:
    json.dump(all_data, out_file, indent=1)

In [26]:
!head ../dataset/French_ELTEC_NER_Open_Dataset.json

{
 "FRA01203_Feval": [
  "I -- La chanson\nIl n'y a pas encore bien longtemps, le voyageur qui allait de Paris \u00e0 Brest, de la capitale du royaume \u00e0 la premi\u00e8re de nos cit\u00e9s maritimes, s'endormait et s'\u00e9veillait deux fois, berc\u00e9 par les cahots de la diligence, avant d'apercevoir les maigres moissons, les pommiers trapus et les ch\u00eanes \u00e9branl\u00e9s de la pauvre Bretagne. Il s'\u00e9veillait la premi\u00e8re fois dans les fertiles plaines du Perche, tout pr\u00e8s de la Beauce, ce paradis des n\u00e9gociants en farine\u00a0: il se rendormait poursuivi par l'aigrelet parfum du cidre de l'Orne et par le patois nasillard des naturels de la Basse-Normandie. Le lendemain matin, le paysage avait chang\u00e9\u00a0; c'\u00e9tait Vitr\u00e9, la gothique momie, qui penche ses maisons noires et les ruines chevelues de son ch\u00e2teau sur la pente raide de sa colline\u00a0; c'\u00e9tait l'\u00e9chiquier de prairies plant\u00e9es \u00e7\u00e0 et l\u00e0 de saul